# Chapter 5. Recommendation Systems

Recommendation systems find their natural application whenever a user is exposed to a wide choice of products or services that they cannot evaluate in a reasonable timeframe. These engines are an important part of an e-commerce business because they assist the clients on the web to facilitate the task of deciding the appropriate items to buy or choose over a large number of candidates not relevant to the end user. Typical examples are Amazon, Netflix, eBay, and Google Play stores that suggest each user the items they may like to buy using the historical data they have collected. Different techniques have been developed in the past 20 years and we will focus on the most important (and employed) methods used in the industry to date, specifying the advantages and disadvantages that characterize each of these methods. The recommendation systems are classified in Content-based Filtering (CBF) and Collaborative Filtering (CF) techniques and other different approaches (association rules, the log-likelihood method, and hybrid methods) will be discussed together with different ways to evaluate their accuracy. The methods will be tested on the MovieLens database (from http://grouplens.org/datasets/movielens/) consisting of 100,000 movie ratings (1 to 5 values) from 943 users on 1,682 movies. Each user has at least 20 ratings and each movie has a list of genres that it belongs to. All the codes shown in this chapter are available, as usual, at https://github.com/ai2010/machine_learning_for_the_web/tree/master/chapter_5 in the rec_sys_methods.ipynb file.

We will start by introducing the main matrix used to arrange the dataset employed by the recommendation system and the metric measures typically used before starting to discuss the algorithms in the following sections.

# Utility matrix

The data used in a recommendation system is divided in two categories: the users and the items. Each user likes certain items, and the rating value rij (from 1 to 5) is the data associated with each user i and item j and represents how much the user appreciates the item. These rating values are collected in matrix, called utility matrix R, in which each row i represents the list of rated items for user i while each column j lists all the users who have rated item j. In our case, the data folder ml-100k contains a file called u.data (and also u.item with the list of movie titles) that has been converted into a Pandas DataFrame (and saved into a csv, utilitymatrix.csv) by the following script:
<img src="./picture/B05143_05_01.jpg" width=600 />
Utility matrix
The output of the first two lines is as follows:
<img src="./picture/B05143_05_02.jpg" width=600 />
Utility matrix
Each column name, apart from the first (which is the user id), defines the name of the movie and the ID of the movie in the MovieLens database (separated by a semicolon). The 0 values represent the missing values and we expect to have a large number of them because the users evaluated far fewer than 1,600 movies. Note that the movies with less than 50 ratings have been removed from the utility matrix, so the number of columns is 604 (603 movies rated more than 50 times). The goal of the recommendation system is to predict these values, but for some techniques to work properly it will be necessary for us to initially set these values (imputation). Usually, two imputation approaches are used: ratings average per user or ratings average per item, and both of them are implemented in the following function:
<img src="./picture/B05143_05_03.jpg" width=600 />
Utility matrix
This function will be called by many of the algorithms implemented in this chapter, so we decided to discuss it here as a reference for future use. Furthermore, in this chapter the utility matrix R will have dimensions N×M with N number of users and M number of items. Due to the recurrent use of the similarity measures by different algorithms, we will define the most commonly used definitions hereafter.

# Similarities measures
In order to compute similarity s between two different vectors x and y, which can be users (rows of utility matrix) or items (columns of utility matrix), two measures are typically used:
<img src="./picture/B05143_05_28.jpg" width=300 />
- Cosine similarity: Similarities measures
<img src="./picture/B05143_05_29.jpg" width=300 />
- Pearson correlation: Similarities measures, where x and y are the averages of the two vectors.
Note that the two measures coincide if the average is 0. We can now start discussing the different algorithms, starting from the CF category. The following sim() function will be used to evaluate the similarity between two vectors:
<img src="./picture/B05143_05_04.jpg" width=600 />
Similarities measures
The SciPy library has been used to compute both similarities (note that the cosine scipy definition is the opposite of what has been defined previously, so the value is subtracted from 1).

# Collaborative Filtering methods
This class of methods is based on the idea that any user will like items appreciated by other users similar to them. In simple terms, the fundamental hypothesis is that a user A, who is similar to user B, will likely rate an item as B did rather than in another way. In practice, this concept is implemented by either comparing the taste of different user's and inferring the future rating for a given user using the most similar users taste (memory-based) or by extracting some rating patterns from what the users like (model-based) and trying to predict the future rating following these patterns. All these methods require a large amount of data to work because the recommendations to a given user rely on how many similar users can be found in the data. This problem is called cold start and it is very well studied in literature, which usually suggests using some hybrid method between CF and CBF to overcome the issue. In our MovieLens database example we assume we have enough data to avoid the cold start problem. Other common problems of CF algorithms are the scalability, because the computation grows with the number of users and products (it may be necessary some parallelization technique), and the sparsity of the utility matrix due to small number of items that any user usually rates (imputation is usually an attempt to handle the problem).

# Memory-based Collaborative Filtering
This subclass employs the utility matrix to calculate either the similarity between users or items. The methods suffer from scalability and cold start issues, but when they are applied to a large or too small utility matrix, they are currently used in many commercial systems today. We are going to discuss user-based Collaborative Filtering and iteFiased Collaborative Filtering hereafter.

# USER-BASED COLLABORATIVE FILTERING
The approach uses a k-NN method (see Chapter 3, Supervised Machine Learning) to find the users whose past ratings are similar to the ratings of the chosen user so that their ratings can be combined in a weighted average to return the current user's missing ratings.

The algorithm is as follows:

For any given user i and item not yet rated j:

- Find the K that is most similar users that have rate j using a similarity metric s.
- Calculate the predicted rating for each item j not yet rated by i as a weighted average over the ratings of the users K:

<img src="./picture/B05143_05_30.jpg" width=300 />
User-based Collaborative Filtering

<img src="./picture/B05143_05_31.jpg" width=100 /> are the average ratings for users i and k to compensate for subjective judgment (some users are generous and some are picky) and s(i, k) is the similarity metric, as seen in the previous paragraph. Note that we can even normalize by the spread of the ratings per user to compare more homogeneous ratings:
<img src="./picture/B05143_05_32.jpg" width=300 />
User-based Collaborative Filtering
Here, σi and σk are the standard deviations of ratings of users i and k.

This algorithm has as an input parameter, the number of neighbors, K but usually a value between 20 and 50 is sufficient in most applications. The Pearson correlation has been found to return better results than cosine similarity, probably because the subtraction of the user ratings means that the correlation formula makes the users more comparable. The following code is used to predict the missing ratings of each user.

The u_vec represents the user ratings values from which the most similar other users K are found by the function FindKNeighbours. CalcRating just computes the predicted rating using the formula discussed earlier (without the spreading correction). Note that in case the utility matrix is so sparse that no neighbors are found, the mean rating of the user is predicted. It may happen that the predicted rating is beyond 5 or below 1, so in such situations the predicted rating is set to 5 or 1 respectively.
<img src="./picture/B05143_05_05.jpg" width=600 />
User-based Collaborative Filtering
# ITEM-BASED COLLABORATIVE FILTERING
This approach is conceptually the same as user-based CF except that the similarity is calculated on the items rather than the users. Since most of the time the number of users can become much larger than the number of items, this method offers a more scalable recommendation system because the items' similarities can be precomputed and they will not change much when new users arrive (if the number of users N is significantly large).

The algorithm for each user i and item j is as follows:

- Find the K most similar items using a similarity metric s that i has already rated.
- Calculate the predicted rating as a weighted average of the ratings of the K items:

<img src="./picture/B05143_05_33.jpg" width=300 />
Item-based Collaborative Filtering
Note that the similarity metric may have a negative value, so we need to restrict the summation to only positive similarities in order to have meaningful (that is, positive) Pij (the relative ordering of items will be correct anyway if we are only interested in the best item to recommend instead of the ratings). Even in this case, a K value between 20 and 50 is usually fine in most applications.

The algorithm is implemented using a class, as follows:
<img src="./picture/B05143_05_06.jpg" width=600 />
Item-based Collaborative Filtering
The constructor of the class CF_itembased calculates the item similarity matrix simmatrix to use any time we want to evaluate missing ratings for a user through the function CalcRatings. The function GetKSimItemsperUser finds K: most similar users to the chosen user (given by u_vec) and CalcRating just implements the weighted average rating calculations discussed previously. Note that in case no neighbors are found, the rating is set to the average or the item's ratings.

# SIMPLEST ITEM-BASED COLLABORATIVE FILTERING – SLOPE ONE
Instead of computing the similarity using the metric discussed previously, a very simple but effective method can be used. We can compute a matrix D in which each entry dij is the average difference between the ratings of items i and j:
<img src="./picture/B05143_05_34.jpg" width=300 />
Simplest item-based Collaborative Filtering – slope one
Here,<img src="./picture/B05143_05_35.jpg" width=300 /> Simplest item-based Collaborative Filtering – slope one is a variable that counts if the user k has rated both i and j items, so <img src="./picture/B05143_05_36.jpg" width=300 />Simplest item-based Collaborative Filtering – slope one is the number of users who have rated both i and j items.

Then the algorithm is as explained in the Item-based Collaborative Filtering section. For each user i and item j:

- Find the K items with the smallest differences from j, Simplest item-based Collaborative Filtering – slope one (the * indicates the possible index values, but for simplicity we relabel them from 1 to K).
<img src="./picture/B05143_05_37.jpg" width=300 />

- Compute the predicted rating as a weighted average:
<img src="./picture/B05143_05_38.jpg" width=300 />
Simplest item-based Collaborative Filtering – slope one
Although this algorithm is much simpler than the other CF algorithms, it often matches their accuracy, is computationally less expensive, and is easy to implement. The implementation is very similar to the class used for item-based CF:
<img src="./picture/B05143_05_07.jpg" width=600 />
Simplest item-based Collaborative Filtering – slope one
The only difference is the matrix: now difmatrix is used to calculate the differences d(i, j) between items i, j, as explained earlier, and the function GetKSimItemsperUser now looks for the smallest difmatrix values to determine the K nearest neighbors. Since it is possible (although unlikely) that two items have not been rated by at least one user, difmatrix can have undefined values that are set to 1000 by default. Note that it is also possible that the predicted rating is beyond 5 or below 1, so in such situations the predicted rating must be set to 5 or 1 appropriately.

# Model-based Collaborative Filtering
This class of methods uses the utility matrix to generate a model to extract the pattern of how the users rate the items. The pattern model returns the predicted ratings, filling or approximating the original matrix (matrix factorization).Various models have been studied in the literature and we will discuss particular matrix factorization algorithms—the Singular Value Decomposition (SVD, also with expectation maximization), the Alternating Least Square (ALS), the Stochastic Gradient Descent (SGD), and the general Non-negative matrix factorization (NMF) class of algorithms.

# ALTERNATIVE LEAST SQUARE (ALS)
This is the simplest method to factorize the matrix R. Each user and each item can be represented in a feature space of dimension K so that:
<img src="./picture/B05143_05_39.jpg" width=300 />
Alternative least square (ALS)
Here, P N×K is the new matrix of users in the feature space, and Q M×K is the projection of the items in the same space. So the problem is reduced to minimize a regularized cost function J:
<img src="./picture/B05143_05_40.jpg" width=300 />
Alternative least square (ALS)
Here, λ is the regularization parameter, which is useful to avoid overfitting by penalizing the learned parameters and ensuring that the magnitudes of the vectors pi and qTj are not too large. The matrix entries Mcij are needed to check that the pair of user i and item j are actually rated, so Mcij is 1 if rij>0, and it's 0 otherwise. Setting the derivatives of J to 0 for each user vector pi and item vector qj, we obtain the following two equations:
<img src="./picture/B05143_05_41.jpg" width=300 />
<img src="./picture/B05143_05_42.jpg" width=300 />
Alternative least square (ALS)
Alternative least square (ALS)
Here Ri and Mci refer to the row i of the matrices R and Mc, and Rj and Mcj refer to the column j of the matrices Mc and R. Alternating the fixing of the matrix P, Q, the previous equations can be solved directly using a least square algorithm and the following function implements the ALS algorithm in Python:
<img src="./picture/B05143_05_08.jpg" width=600 />
Alternative least square (ALS)
The matrix Mc is called mask, the variable l represents the regularization parameter lambda and is set to 0.001 by default, and the least square problem has been solved using the linalg.solve function of the Numpy library. This method usually is less precise than both Stochastic Gradient Descent (SGD) and Singular Value Decomposition (SVD) (see the following sections) but it is very easy to implement and easy to parallelize (so it can be fast).

# STOCHASTIC GRADIENT DESCENT (SGD)
This method also belongs to the matrix factorization subclass because it relies on the approximation of the utility matrix R as:
<img src="./picture/B05143_05_43.jpg" width=300 />
Stochastic gradient descent (SGD)
Here, the matrices P(N×K) and Q(M×K) represent the users and the items in a latent feature space of K dimensions. Each approximated rating Stochastic gradient descent (SGD) can be expressed as follows:
<img src="./picture/B05143_05_45.jpg" width=300 />
Stochastic gradient descent (SGD)
The matrix R^ is found, solving the minimization problem of the regularized squared errors e2ij as with the ALS method (cost function J as in Chapter 3, Supervised Machine Learning):
<img src="./picture/B05143_05_47.jpg" width=300 />
Stochastic gradient descent (SGD)
This minimization problem is solved using the gradient descent (see Chapter 3, Supervised Machine Learning):
<img src="./picture/B05143_05_48.jpg" width=300 />
<img src="./picture/B05143_05_49.jpg" width=300 />
Stochastic gradient descent (SGD)
Stochastic gradient descent (SGD)
Here, α is the learning rate (see Chapter 3, Supervised Machine Learning) and <img src="./picture/B05143_05_50.jpg" width=300 />Stochastic gradient descent (SGD). The technique finds R alternating between the two previous equations (fixing qkj and solving Pik, and vice versa) until convergence. SGD is usually easier to parallelize (so it can be faster) than SVD (see the following section) but is less precise at finding good ratings. The implementation in Python of this method is given by the following script:
<img src="./picture/B05143_05_09.jpg" width=600 />
Stochastic gradient descent (SGD)
This SGD function has default parameters that are learning rate α = 0.0001, regularization parameter λ = l =0.001, maximum number of iterations 1000, and convergence tolerance tol = 0.001. Note also that the items not rated (0 rating values) are not considered in the computation, so an initial filling (imputation) is not necessary when using this method.

# NON-NEGATIVE MATRIX FACTORIZATION (NMF)
This is a group of methods that finds the decomposition of the matrix R again as a product of two matrices P (N×K) and Q (M×K) (where K is a dimension of the feature space), but their elements are required to be non-negative. The general minimization problem is as follows:
<img src="./picture/B05143_05_51.jpg" width=300 />
Non-negative matrix factorization (NMF)
Here, α is a parameter that defines which regularization term to use (0 squared, 1 a lasso regularization, or a mixture of them) and λ is the regularization parameter. Several techniques have been developed to solve this problem, such as projected gradient, coordinate descent, and non-negativity constrained least squares. It is beyond the scope of this book to discuss the details of these techniques, but we are going to use the coordinate descent method implemented in sklearn NFM wrapped in the following function:
<img src="./picture/B05143_05_10.jpg" width=600 />
Non-negative matrix factorization (NMF)
Note that an imputation may be performed before the actual factorization takes place and that the function fit_transform returns the P matrix while the QT matrix is stored in the nmf.components_ object. The α value is assumed to be 0 (squared regularization) and λ = l =0.01 by default. Since the utility matrix has positive values (ratings), this class of methods is certainly a good fit to predict these values.

# SINGULAR VALUE DECOMPOSITION (SVD)
We have already discussed this algorithm in Chapter 2, Unsupervised Machine Learning, as a dimensionality reduction technique to approximate a matrix by decomposition into matrices U, Σ, V (you should read the related section in Chapter 2, Unsupervised Machine Learning, for further technical details). In this case, SVD is used as a matrix factorization technique, but an imputation method is required to initially estimate the missing data for each user; typically, the average of each utility matrix row (or column) or a combination of both (instead of leaving the zero values) is used. Apart from directly applying the SVD to the utility matrix, another algorithm that exploits an expectation-maximization (see Chapter 2, Unsupervised Machine Learning) can be used as follows, starting from the matrix Singular value decomposition (SVD):<img src="./picture/B05143_05_52.jpg" width=300 />

m-step: Perform <img src="./picture/B05143_05_53.jpg" width=300 />Singular value decomposition (SVD)
e-step: <img src="./picture/B05143_05_54.jpg" width=300 />Singular value decomposition (SVD)
This procedure is repeated until the sum of squared errors <img src="./picture/B05143_05_55.jpg" width=300 />Singular value decomposition (SVD) is less than a chosen tolerance. The code that implements this algorithm and the simple SVD factorization is as follows:
<img src="./picture/B05143_05_11.jpg" width=600 />

Singular value decomposition (SVD)
Note that the SVD is given by the sklearn library and both imputation average methods (user ratings' average and item ratings' average) have been implemented, although the function default is none, which means that the zero values are left as initial values. For the expect-maximization SVD, the other default parameters are the convergence tolerance (0.0001) and the maximum number of iterations (10,000). This method (especially with expectation-maximization) is slower than the ALS, but the accuracy is generally higher. Also note that the SVD method decomposes the utility matrix subtracted by the user ratings' mean since this approach usually performs better (the user ratings' mean is then added after the SVD matrix has been computed).

We finish remarking that SVD factorization can also be used in memory-based CF to compare users or items in the reduced space (matrix U or VT) and then the ratings are taken from the original utility matrix (SVD with k-NN approach).

# CBF methods
This class of method relies on the data that describes the items, which is then used to extract the features of the users. In our MovieLens example, each movie j has a set of G binary fields to indicate if it belongs to one of the following genres: unknown, action, adventure, animation, children's, comedy, crime, documentary, drama, fantasy, film noir, horror, musical, mystery, romance, sci-fi, thriller, war, or western.

Based on these features (genres), each movie is described by a binary vector mj with G dimensions (number of movie genres) with entries equal to 1 for all the genres contained in movie j, or 0 otherwise. Given the dataframe that stores the utility matrix called dfout in the Utility matrix section mentioned earlier, these binary vectors mj are collected from the MoviesLens database into a dataframe using the following script:
<img src="./picture/B05143_05_12.jpg" width=600 />
CBF methods
The movies content matrix has been saved in the movies_content.csv file ready to be used by the CBF methods.

The goal of the content-based recommendation system is to generate the user's profile with the same fields to indicate how much the user likes each genre. The problem with this method is that the content description of the item is not always available, so it is not always possible to employ this technique in the e-commerce environment. The advantage is that the recommendations to a specific user are independent of the other users' ratings, so it does not suffer from cold start problems due to an insufficient number of users' ratings for particular items. Two approaches are going to be discussed to find the best recommendation methodologies. The first methodology simply generates the user's profile associated with the average ratings of the movies seen by each user to each genre and the cosine similarity is used to find the movies most similar to the user preferences. The second methodology is a regularized linear regression model to generate the user's profile features from the ratings and the movie features so that the ratings of the movies not yet seen by each user can be predicted using these users' profiles.

# Item features average method

The approach is really simple and we are going to explain it using the features that describe the movies in the MovieLens example, as discussed previously. The objective of the method is to generate the movie genres' preferences vector  <img src="./picture/B05143_05_56.jpg" width=300 />Item features average method for each user i (length equal to G). This is done by calculating the average rating $r_i$ and each genre entry g; $v_{ig}$ is given by the sum of ratings of the movies seen by user i (Mi) containing the genre g, minus the average $r_i$and divided by the number of movies containing genre g:
<img src="./picture/B05143_05_59.jpg" width=300 />
Item features average method
Here, Ikg is 1 if the movie k contains genre g; otherwise it is 0.

The vectors $v_i$ are then compared to the binary vectors mj using the cosine similarity and the movies with the highest similarity values are recommended to the user i. The implementation of the method is given by the following Python class:
<img src="./picture/B05143_05_13.jpg" width=600 />
Item features average method
The constructor stores the list of the movie titles in Movieslist and the movie features in the Movies vector, and the GetRecMovies function generates the user genres' preferences vector, that is, Item features average method (applying the preceding formula) called features_u, and returns the most similar items to this vector.

# Regularized linear regression method

The method learns the movie preferences of the users as parameters
<img src="./picture/B05143_05_61.jpg" width=300 />
Regularized linear regression method of a linear model,
<img src="./picture/B05143_05_62.jpg" width=300 />
with Regularized linear regression method, where N is the number of users and G is the number of features (movie genres) of each item. We add an intercept value on the user parameters θi (θi0 = 1) and also the movie vector mj that has the same value mj0=1, and so <img src="./picture/B05143_05_63.jpg" width=300 />Regularized linear regression method. To learn the vectors of parameters qi , we solve the following regularized minimization problem:
<img src="./picture/B05143_05_64.jpg" width=300 />
Regularized linear regression method
Here, Iij is 1; that is, user i watched the movie, otherwise j is 0 and λ is the regularization parameter (see Chapter 3, Supervised Machine Learning).

The solution is given by applying gradient descent (see Chapter 3, Supervised Machine Learning). For each user i:
<img src="./picture/B05143_05_65.jpg" width=300 />
Regularized linear regression method (k=0)
<img src="./picture/B05143_05_66.jpg" width=300 />
Regularized linear regression method (k>0)
Since we are adding 1 entry to the movie and user vectors respectively, the distinction between learning the intercept parameter (k=0) and the others is necessary (there is no possibility of overfitting on the intercept, so no need to regularize on it). After the parameters qi are learned, the recommendation is performed by simply applying for any missing rating rij in the formula Regularized linear regression method.
<img src="./picture/B05143_05_67.jpg" width=300 />
The method is implemented by the following code:
<img src="./picture/B05143_05_14.jpg" width=600 />
Regularized linear regression method
The constructor of the class CBF_regression just performs the gradient descent to find the parameters θi (called Pmatrix) while the function CalcRatings finds the most similar rating vector in the stored utility matrix R (in case the user is not present in the utility matrix) and then it uses the corresponding parameters' vector to predict the missing ratings.

# Association rules for learning recommendation system
Although this method is not used often in many commercial recommendation systems, association rules learning is certainly a method worth knowing about because of historical data reasons, and it can be employed to solve a wide range of problems in real-world examples. The main concept of this method is to find relationships among items based on some statistical measure of the occurrences of the items in the database of transactions T (for example, a transaction could be the movies seen by a user i or the products bought by i). More formally, a rule could be {item1,item2} => {item3}, that is, a set of items ({item1,item2}) implies the presence of another set ({item3}). Two definitions are used to characterize each X=>Y rule:

- Support: Given a set of items X, the support supp(X) is the portion of transactions that contains the set X over the total transactions.
- Confidence: It is the fraction of transactions that contains the set X that also contains the set Y: conf(X=>Y)=supp(X U Y)/supp(X). Note that the confidence conf(X=>Y) can have a very different value than conf(Y=>X).
Support represents the frequency of a certain rule on the transaction database, while the confidence indicates the probability that set Y will occur if set X is present. In other words, the support value is chosen to filter the number of rules we want to mine from the database (the higher the support, the fewer rules will satisfy the condition), while the confidence can be thought of as a similarity metric between sets X and Y. In the case of the movie recommendation system, the transaction database can be generated from the utility matrix R considering the movies each user likes, and we look for rules composed by sets X and Y that contain only one item (movie). These rules are collected in a matrix, ass_matrix, in which each entry ass_matrixij represents the confidence of the rule i =>j. The recommendations for the given user are obtained by simply multiplying the ass_matrix by his ratings u_vec: recitems=uvec내적ass matrix, and sorting all the values recitems by the largest value corresponding to the most recommended movie to the least. Therefore, this method does not predict the ratings, but the list of movie recommendations; however, it is fast and it also works well with a sparse utility matrix. Note that to find all the possible combinations of items to form sets X and Y as fast as possible, two algorithms have been developed in the literature: apriori and fp-growth (not discussed here since we only require rules with one item per set X and Y).

The class that implements the method is as follows:
<img src="./picture/B05143_05_15.jpg" width=600 />
Association rules for learning recommendation system
The class constructor takes as input parameters the utility matrix Umatrix, the movie titles list Movieslist, the support min_support, confidence min_confidence thresholds (default 0.1), and the likethreshold, which is the minimum rating value to consider a movie in a transaction (default 3). The function combine_lists finds all the possible rules, while filterSet just reduces the rules to the subset that satisfies the minimum support threshold. calc_confidence_matrix fills the ass_matrix with the confidence value that satisfies the minimum threshold (otherwise 0 is set by default) and GetRecItems returns the list of recommended movies given the user ratings u_vec.

# log-likelihood ratios recommendation system method
The log-likelihood ratio (LLR) is a measure of how two events A and B are unlikely to be independent but occur together more than by chance (more than the single event frequency). In other words, the LLR indicates where a significant co-occurrence might exist between two events A and B with a frequency higher than a normal distribution (over the two events variables) would predict.

It has been shown by Ted Dunning (http://tdunning.blogspot.it/2008/03/surprise-and-coincidence.html) that the LLR can be expressed based on binomial distributions for events A and B using a matrix k with the following entries:
<img src="./picture/캡처.jpg" width=600 />

Log-likelihood ratios recommendation system method
Here, Log-likelihood ratios recommendation system method and Log-likelihood ratios recommendation system method is the Shannon entropy that measures the information contained in the vector p.

Note: Log-likelihood ratios recommendation system method is also called the Mutual Information (MI) of the two event variables A and B, measuring how the occurrence of the two events depend on each other.

This test is also called G2, and it has been proven effective to detect co-occurrence of rare events (especially in text analysis), so it's useful with sparse databases (or a utility matrix, in our case).

In our case, the events A and B are the like or dislike of two movies A and B by a user, where the event of like a movie is defined when the rating is greater than 3 (and vice versa for dislike). Therefore, the implementation of the algorithm is given by the following class:

Log-likelihood ratios recommendation system method
The constructor takes as input the utility matrix, the movie titles list, and the likethreshold that is used to define if a user likes a movie or not (default 3). The function loglikelihood_ratio generates the matrix with all the LLR values for each pair of movies i and j calculating the matrix k (calc_k) and the corresponding LLR (calc_llr). The function GetRecItems returns the recommended movie list for the user with ratings given by u_vec (the method does not predict the rating values).

<img src="./picture/B05143_05_16.jpg" width=600 />

# Hybrid recommendation systems
This is a class of methods that combine both CBF and CF in a single recommender to achieve better results. Several approaches have been tried and can be summarized in the following categories:

- Weighted: The CBF and CF predicted ratings are combined in to some weighted mean.
- Mixed: CF and CBF predicted movies are found separately and then merged in to a single list.
- Switched: Based on certain criteria, the CF predictions or CBF predictions are used.
- Feature combination: CF and CBF features are considered together to find the most similar users or items.
- Feature augmentation: Similar to feature combination, but the additional features are used to predict some ratings and then the main recommender uses these ratings to produce the recommendation list. For example, Content-Boosted Collaborative Filtering learns the ratings of unrated movies by a content-based model and then a collaborative approach is employed to define the recommendations.


As an example, we implement two hybrid feature combination methods merging an item's features CBF method with a user-based CF method. The first method employs a user-based CF to the expanded utility matrix that now also contains the average rating per genre per user. The Python class is as follows:
<img src="./picture/B05143_05_17.jpg" width=600 />
<img src="./picture/B05143_05_18.jpg" width=600 />
Hybrid recommendation systems
Hybrid recommendation systems
The constructor generates the expanded utility matrix with the movies' genres average rating features associated to each user, Umatrix_mfeats. The function CalcRatings finds the K-NN using the Pearson correlation comparing the expanded feature vectors of the users. The second method applies and SVD factorization to the expanded utility matrix that contains the genre preferences for each user.
<img src="./picture/B05143_05_19.jpg" width=600 />
Hybrid recommendation systems
As the SVD method, the ratings are subtracted with the user rating's average, and genre preferences are subtracted from the same user rating's average.

# Evaluation of the recommendation systems
We have discussed all of the most relevant methods used in the commercial environment to date. The evaluation of a recommendation system can be executed offline (using only the data in the utility matrix) or online (using the utility matrix data and the new data provided in real time by each user using the website). The online evaluation procedures are discussed in Chapter 7, Movie Recommendation System Web Application, together with a proper online movie recommendation system website. In this section, we will evaluate the performances of the methods using two offline tests often used to evaluate recommendation systems: root mean square error on ratings and ranking accuracy. For all the evaluations in which k-fold cross-validation (see Chapter 3, Supervised Machine Learning) is applicable, a 5-fold cross-validation has been performed to obtain more objective results. The utility matrix has been divided in to 5 folds using the following function:
<img src="./picture/B05143_05_20.jpg" width=600 />
Evaluation of the recommendation systems
Here df is a data frame object that stores the utility matrix and k is the number of folds. In the validation set, for each user ratings' vector u_vec, half of the ratings have been hidden so that the real value can be predicted.
<img src="./picture/B05143_05_21.jpg" width=600 />
Evaluation of the recommendation systems
u_vals stores the values to predict while u_test contains the ratings for testing the algorithms. Before we start to compare the different algorithms with the different measures, we load the utility matrix and the movie content matrix into data frames and split the data into 5 folds for cross-validation.
<img src="./picture/B05143_05_22.jpg" width=600 />
Evaluation of the recommendation systems
df_vals contains the validation sets so the HideRandomRatings function presented in this section needs to be applied.
<img src="./picture/B05143_05_23.jpg" width=600 />
Evaluation of the recommendation systems
The data available in the movies matrix, the movieslist list, and the data frames df_trains, vals_vecs_folds, tests_vecs_folds are now ready to be used for training and validating all the methods discussed in the previous sections. We can start evaluating the root mean square error (RMSE).

# Root mean square error (RMSE) evaluation
This validation technique is applicable only on CF methods and linear regression CBF since the predicted ratings are generated only by these algorithms. Given each rating rij in u_vals in the validation sets, the predicted rating $HATr_{ij}$ is calculated using each method and the root mean square error is obtained:

RMSE = <img src="./picture/B05143_05_75.jpg" width=300 />Root mean square error (RMSE) evaluation

Here, Nval is the number of ratings in the u_vals vectors. The presence of the square factor in this formula highly penalizes the large errors, so the methods with low RMSE (best values) are characterized by small errors spread over all the predicted ratings instead of large errors on few ratings, like the mean absolute error MAE=<img src="./picture/B05143_05_76.jpg" width=300 />Root mean square error (RMSE) evaluation would prefer.

The code to calculate the RMSE for the memory-based CF user-based and item-based methods is as follows:
<img src="./picture/B05143_05_24.jpg" width=600 />
<img src="./picture/B05143_05_25.jpg" width=600 />
Root mean square error (RMSE) evaluation
Root mean square error (RMSE) evaluation
For each method, the SE function is called to compute the error for each fold and then the total RMSE of the folds is obtained.

Using 5 nearest-neighbors for item-based CF with slope one and 20 for user-based CF, the methods have the following errors:

<img src="./picture/캡처1.jpg" width=600 />

All have similar RMSE values but the best method is item-based Collaborative Filtering.

For the model-based methods, instead of not hidden validation ratings, u_test are included in the utility matrix for training and then the RMSE is calculated using the following script:
<img src="./picture/B05143_05_26.jpg" width=600 />
Root mean square error (RMSE) evaluation
The code calculates the RMSE only for CBF regression and SVD, and the reader can easily replicate the code to calculate the error for the other algorithms since most of the required code is just commented (SVD expect-maximization, SGD, ALS, and NMF). The results are shown in the following table (K dimension feature space):
<img src="./picture/캡처2.jpg" width=600 />
As expected, the ALS and SGD are the worst methods but they are discussed because they are instructive from a didactic point of view (they are also slow because the implementation is not as optimized as the methods from sklearn library).

All the others have similar results. However, just note that the hybrid methods have slightly better results than the corresponding SVD and CF user-based algorithms. Note that the movies to predict are chosen randomly so the results may vary.

# Classification metrics
The rating error RMSE does not really indicate the quality of a method but is an academic measure that is not really used in a commercial environment. The goal of a website is to present content that is relevant to the user regardless of the exact rating the user gives. In order to evaluate the relevance of the recommended items, the precision, recall, and f1 (see Chapter 2, Unsupervised Machine Learning) measures are used where the correct predictions are the items with ratings greater than 3. These measures are calculated on the first 50 items returned by each algorithm (if the algorithm return a recommended list or the 50 items with the highest predicted ratings for the other methods). The function that calculates the measures is as follows:
<img src="./picture/B05143_05_27.jpg" width=600 />
Classification metrics
Here, Boolean ratingsval indicates if the method returns ratings or recommended list. We use the function ClassificationMetrics in the same way we compute the RMSE for all the methods, so the actual code to evaluate the measures is not shown (you can write it as an exercise). The following table summarizes the results for all the methods (neighs is number of nearest-neighbors, K dimension feature space):
<img src="./picture/캡처3.jpg" width=600 />
From the results you can see that the best method is association rules, and there is good precision also for the LLR, hybrid CBFCF user-based, and CF user-based methods. Note that the results may vary since the movies to predict have been randomly chosen.

# Summary
In this chapter, we discussed the most commonly used recommendation system methods from Collaborative Filtering and content-based filtering to two simple hybrid algorithms. Note also that in the literature are present modal recommendation systems in which different data (user gender, demographics, views, locations, devices, and so on) are incorporated in to the same algorithm. These methods are more advanced and more different data is needed to use them.

In Chapter 7, Movie Recommendation System Web Application, we will implement a web recommendation system using the methods discussed in this chapter, but before that we will present the Django framework to build web applications in Chapter 6, Getting Started with Django.